# 09: Long-Term Memory & Contextual Learning 🧠

This notebook demonstrates the **Cognitive Layer** of the SalesOps Agent Suite (Day 8).

Previously, our agents were "amnesic"—they processed every anomaly as if it were the first one they had ever seen. In this notebook, we introduce the **Memory Bank**, a vector-based semantic storage system that allows the agent to "remember" past incidents and "learn" from them.

### 🎯 Goals
1.  **Initialize Memory Bank:** Spin up the local vector store with PII protection enabled.
2.  **Simulate Learning:** "Teach" the agent by feeding it historical resolutions (e.g., "We fixed the West Region spike by updating the pricing model").
3.  **Demonstrate RAG (Retrieval Augmented Generation):** Feed a *new* anomaly to the Explainer Agent and verify that it automatically retrieves and references the relevant history to provide a smarter answer.
4.  **Verify Safety:** Prove that sensitive data (emails/phones) is automatically redacted before being stored.

### 🏗️ Components Used
* `agents.memory_agent.MemoryAgent`: The bridge between business logic and the vector store.
* `memory.memory_bank.MemoryBank`: The core storage engine handling Embeddings, TTL, and Atomicity.
* `agents.anomaly_llm_agent.AnomalyExplainerAgent`: The AI analyst now upgraded with RAG capabilities.

## 1: Imports

In [1]:
import sys
import os
import json
import time
from dotenv import load_dotenv

# Add project root
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

os.environ["OBSERVABILITY_DIR"] = os.path.join(project_root, "outputs", "observability")

load_dotenv()

from agents.memory_agent import MemoryAgent
from agents.anomaly_llm_agent import AnomalyExplainerAgent

print("✅ Agents Loaded")

c:\Users\YASH\anaconda3\envs\salesops\Lib\site-packages\google\cloud\aiplatform\models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


✅ Agents Loaded


## 2: Teach the Agent (Simulated Run 1)

In [2]:
# 1. Create a fake previous anomaly that was resolved
history_event = {
    "entity_id": "Technology",
    "metric": "Sales",
    "score": 5.5,
    "explanation_short": "Spike caused by bulk laptop order from Acme Corp.",
    "type": "resolution",
}
action_taken = {"type": "create_ticket"}

# 2. Store it in Memory
mem_agent = MemoryAgent()
mem_agent.remember_anomaly_resolution(history_event, action_taken)

print("🧠 Agent has learned from a past Technology Sales spike.")

🧠 Agent has learned from a past Technology Sales spike.


## 3: New Anomaly (Simulated Run 2)

In [3]:
# 3. Create a NEW similar anomaly
current_anomaly = {
    "anomaly_id": "new_event_123",
    "entity_id": "Technology",
    "level": "category",
    "metric": "Sales",
    "value": 25000,
    "expected": 5000,
    "score": 4.8,
    "context": {"window_mean": 5000},
    "detector": "zscore",
}

# 4. Explain it (This triggers the RAG lookup)
explainer = AnomalyExplainerAgent()
print("🤖 Asking Gemini to explain the new anomaly...")

result = explainer.batch_explain([current_anomaly])

🤖 Asking Gemini to explain the new anomaly...


## 4: View Results

In [4]:
from IPython.display import display, Markdown

rec = result[0]
md = f"""
### 🧠 RAG Result
**Explanation:** {rec['explanation_full']}

**Did it use memory?**
Check if the explanation mentions "Acme Corp" or similar past events.
"""
display(Markdown(md))


### 🧠 RAG Result
**Explanation:** The sales value of 25,000.00 is 5 times the expected value of 5,000.00, as indicated by the score of 4.80 against a historical window mean of 5000. Historical data shows similar spikes in Technology sales have been attributed to large bulk orders, such as one from Acme Corp., prompting ticket creation.

**Did it use memory?**
Check if the explanation mentions "Acme Corp" or similar past events.


---
## ⏭️ Next Step: Opening the "Black Box" (Observability)

Success! We have upgraded our agent with **Long-Term Semantic Memory**.
* It can **Recall** history using Vector Search.
* It can **Learn** from resolutions.
* It respects **Privacy** (PII Redaction) and **Storage Limits** (TTL).

**However, we have created a "Black Box."**
We now have Orchestrators, Parallel Threads, Vector Databases, and LLM calls all happening at once. If something runs slow or fails, **we don't know why.**

* Was it the Vector DB query?
* Was it the Gemini API latency?
* Did the retry logic trigger?

In **Day 9**, we will build the **Observability Layer**.
We will implement **Structured Logging, Distributed Tracing, and Metrics** to create an "X-Ray" view of our system, proving to the judges that we are production-ready.

👉 **Proceed to `notebooks/10_observability_dashboard.ipynb`.**